In [8]:
import akshare as ak
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

import db

# code_list = ak.stock_info_a_code_name()
df = pd.DataFrame(columns=['total_val', 'stock_id', 'category', 'stock_name'])
# print(code_list['code'][0])
stock_individual_info_em_df = ak.stock_individual_info_em(symbol="000001")
df_transposed = stock_individual_info_em_df.set_index('item').T.reset_index()
df_transposed = df_transposed.rename(columns={
    '总市值': 'total_val',
    '行业': 'category',
    '股票代码': 'stock_id',
    '股票简称': 'stock_name'
})

df = pd.concat([df, df_transposed[['total_val', 'stock_id', 'category', 'stock_name']]], ignore_index=True)


db_username = 'root'
db_password = 'qwertyui'
db_host = 'localhost'  # 或者其他主机地址
db_port = '3306'
db_name = 'schemas'
table_name = 'stock_info'  # 要创建或写入的表的名称

# 创建数据库引擎
engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

# 将数据写入 MySQL
df.to_sql(table_name, engine, if_exists='replace', index=False)

# 关闭数据库连接
engine.dispose()

In [26]:
import akshare as ak
from datetime import datetime
import pandas as pd
now = datetime.now()
data = ak.stock_zh_a_hist_min_em(symbol="000001", period='1', adjust='')
data.rename(columns={
                '时间': 'date',
                '开盘': 'open_price',
                '收盘': 'close_price',
                '最高': 'high_price',
                '最低': 'low_price',
                '成交量': 'volume',
                '成交额': 'turnover',
                '最新价': 'now_price'
            }, inplace=True)
data['date'] = pd.to_datetime(data['date'])
print(now.date())
data = data[data['date'].dt.date == now.date()]
print(data)

2023-11-27
                    date  open_price  close_price  high_price  low_price  \
964  2023-11-27 09:30:00       10.09        10.09       10.09      10.09   
965  2023-11-27 09:31:00       10.09        10.05       10.09      10.05   
966  2023-11-27 09:32:00       10.06        10.05       10.06      10.04   
967  2023-11-27 09:33:00       10.05        10.03       10.05      10.02   
968  2023-11-27 09:34:00       10.02        10.02       10.03      10.01   
...                  ...         ...          ...         ...        ...   
1043 2023-11-27 10:49:00        9.99         9.99        9.99       9.98   
1044 2023-11-27 10:50:00        9.99         9.99        9.99       9.98   
1045 2023-11-27 10:51:00        9.98         9.99        9.99       9.98   
1046 2023-11-27 10:52:00        9.98         9.98        9.99       9.98   
1047 2023-11-27 10:53:00        9.98         9.98        9.99       9.98   

      volume    turnover  now_price  
964     7353   7419177.0     10.090  


In [39]:
import pandas as pd
import pymysql
from datetime import datetime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import akshare as ak

# 获取数据
data = ak.stock_bid_ask_em(symbol="000001")
data_transposed = data.set_index('item').T.reset_index()
data_transposed['stock_id'] = "000001"
data_transposed['time'] = datetime.now().timestamp()
data_transposed.drop(columns='index', inplace=True)

# 数据库配置
db_username = 'root'
db_password = 'qwertyui'
db_host = 'localhost'
db_port = '3306'
db_name = 'schemas'
table_name = 'stock_bid'

# 创建数据库连接
engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

# 创建会话
Session = sessionmaker(bind=engine)
session = Session()

# 获取数据并处理
data = ak.stock_bid_ask_em(symbol="000001")
data_transposed = data.set_index('item').T.reset_index()
data_transposed['stock_id'] = "000001"
data_transposed['time'] = datetime.now().timestamp()
data_transposed.drop(columns='index', inplace=True)

# 执行删除和插入操作
for index, row in data_transposed.iterrows():
    # 查询是否存在
    existing_data = session.query(StockBid).filter_by(stock_id=row['stock_id']).first()
    if existing_data:
        session.delete(existing_data)
    new_data = StockBid(stock_id=row['stock_id'], ...)
    session.add(new_data)

session.commit()
session.close()

SyntaxError: positional argument follows keyword argument (331269335.py, line 40)

In [41]:
stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol="002838", period="daily", start_date="20231120", end_date='20231124', adjust="")
print(stock_zh_a_hist_df)

           日期     开盘     收盘     最高     最低    成交量          成交额    振幅   涨跌幅  \
0  2023-11-20  13.63  13.62  13.70  13.49  20731  28155062.00  1.54  0.07   
1  2023-11-21  13.62  13.64  13.80  13.60  23623  32355003.24  1.47  0.15   
2  2023-11-22  13.59  13.51  13.73  13.50  21763  29578591.88  1.69 -0.95   
3  2023-11-23  13.46  13.61  13.64  13.46  19327  26248644.60  1.33  0.74   
4  2023-11-24  13.61  13.49  13.64  13.44  18765  25341356.35  1.47 -0.88   

    涨跌额   换手率  
0  0.01  0.53  
1  0.02  0.61  
2 -0.13  0.56  
3  0.10  0.50  
4 -0.12  0.48  


In [17]:
import db
db.fetch_min_data()

Exception in thread Thread-32:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 954, in _bootstrap_inner
Exception in thread Thread-31:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 954, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 892, in run
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/mac/Downloads/myflask/db.py", line 73, in get_min_data
    with lock:
NameError: name 'engine' is not defined
    self._target(*self._args, **self._kwargs)
  File "/Users/mac/Downloads/myflask/db.py", line 73, in get_min_data
    with lock:
NameError: name 'engine' is not defined
Exception in thread Thread-30:
Traceback (most recent 

In [ ]:
import akshare as ak
import pandas as pd
import threading
from sqlalchemy import create_engine
from datetime import datetime, timedelta
db_username = 'root'
db_password = 'qwertyui'
db_host = 'localhost'
db_port = '3306'
db_name = 'schemas'
table_name = 'stock_info'

# 创建数据库引擎
engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

sql = f"SELECT * FROM {table_name}"


lock = threading.Lock()
monitor = 0


In [45]:
import db
code = "000001"
data = ak.stock_zh_a_hist(symbol=code,period="daily",start_date="20221120",end_date="20231119",adjust="")
data = data.rename(columns={
    '日期': 'date',
    '开盘': 'open_price',
    '收盘': 'close_price',
    '最高': 'high_price',
    '最低': 'low_price',
    '成交量': 'volume',
    '成交额': 'turnover',
    '振幅': 'amplitude',
    '涨跌幅': 'price_change_pct',
    '涨跌额': 'price_change_amt',
    '换手率': 'turnover_rate'
})
data['stock_id']=code
db.pd2sql(data,"stock_hist")


In [8]:
import akshare as ak
stock_intraday_sina_df = ak.stock_intraday_sina(symbol="sz000001", date="20231127")
stock_intraday_sina_df


,symbol,name,ticktime,price,volume,prev_price,kind
0,sz000001,平安银行,09:25:00,10.09,735300,0.00,U
1,sz000001,平安银行,09:30:00,10.09,51900,10.09,U
2,sz000001,平安银行,09:30:03,10.09,197800,10.09,U
3,sz000001,平安银行,09:30:06,10.08,88900,10.09,D
4,sz000001,平安银行,09:30:09,10.08,230500,10.08,D
...,...,...,...,...,...,...,...
717,sz000001,平安银行,14:17:09,10.00,76000,10.00,U
718,sz000001,平安银行,14:17:27,10.00,73700,10.00,D
719,sz000001,平安银行,14:17:42,10.00,22800,9.99,U
720,sz000001,平安银行,14:19:18,10.00,34400,10.00,D


In [25]:
import akshare as ak

import akshare as ak

# 注意：该接口返回的数据只有最近一个交易日的有开盘价，其他日期开盘价为 0
stock_zh_a_hist_min_em_df = ak.stock_zh_a_hist_min_em(symbol="000001", period='1', adjust='')
print(stock_zh_a_hist_min_em_df)


                      时间   开盘     收盘     最高     最低    成交量         成交额     最新价
0    2023-11-22 09:30:00  0.0  10.26  10.26  10.26   3528   3619728.0  10.260
1    2023-11-22 09:31:00  0.0  10.26  10.27  10.24  12808  13141486.0  10.260
2    2023-11-22 09:32:00  0.0  10.26  10.27  10.26   2826   2900229.0  10.260
3    2023-11-22 09:33:00  0.0  10.26  10.27  10.25   2887   2962150.0  10.260
4    2023-11-22 09:34:00  0.0  10.26  10.27  10.26   5017   5148385.0  10.261
..                   ...  ...    ...    ...    ...    ...         ...     ...
959  2023-11-27 14:56:00  0.0  10.02  10.02  10.01   1398   1400295.0  10.001
960  2023-11-27 14:57:00  0.0  10.02  10.02  10.01   3765   3772007.0  10.001
961  2023-11-27 14:58:00  0.0  10.01  10.01  10.01     42     42042.0  10.001
962  2023-11-27 14:59:00  0.0  10.01  10.01  10.01      0         0.0  10.001
963  2023-11-27 15:00:00  0.0  10.01  10.01  10.01   4445   4449695.0  10.001

[964 rows x 8 columns]


In [28]:
import akshare as ak
from datetime import datetime
now = datetime.now()
stock_zh_a_spot_em_df = ak.stock_zh_a_spot_em()
stock_zh_a_spot_em_df = stock_zh_a_spot_em_df.rename(columns={
            '名称': 'stock_name',
            '代码': 'stock_id',
            '今开': 'open_price',
            '最新价': 'close_price',
            '最高': 'high_price',
            '最低': 'low_price',
            '成交量': 'volume',
            '成交额': 'turnover',
            '振幅': 'amplitude',
            '涨跌幅': 'price_change_pct',
            '涨跌额': 'price_change_amt',
            '换手率': 'turnover_rate'
        })
stock_zh_a_spot_em_df = stock_zh_a_spot_em_df.filter([
    'stock_name',
    'stock_id',
    'open_price',
    'close_price',
    'high_price',
    'low_price',
    'volume',
    'turnover',
    'amplitude',
    'price_change_pct',
    'price_change_amt',
    'turnover_rate'
])
date_str = now.strftime('%Y%m%d')  # 日期，格式为“年月日”
time_str = now.strftime('%H:%M')  # 时间，精确到分钟
stock_zh_a_spot_em_df['date'] = date_str
stock_zh_a_spot_em_df['time'] = time_str
print(stock_zh_a_spot_em_df)


     stock_name stock_id  open_price  close_price  high_price  low_price  \
0          N思泰克   301568         NaN        70.00         NaN        NaN   
1          三祥科技   831195         NaN        19.90         NaN        NaN   
2          驰诚股份   834407         NaN        15.37         NaN        NaN   
3          大地电气   870436         NaN        10.50         NaN        NaN   
4          威贸电子   833346         NaN        12.90         NaN        NaN   
...         ...      ...         ...          ...         ...        ...   
5550       开勒股份   301070         NaN        36.20         NaN        NaN   
5551       万德股份   836419         NaN         9.25         NaN        NaN   
5552       纬达光电   873001         NaN         7.12         NaN        NaN   
5553       欧康医药   833230         NaN        10.00         NaN        NaN   
5554       峆一药业   430478         NaN        24.10         NaN        NaN   

      volume  turnover  amplitude  price_change_pct  price_change_amt  \
0        NaN  

In [18]:
from datetime import timedelta,datetime
end_date = datetime.now().strftime("%Y%m%d")
start_date = (datetime.now() - timedelta(days=1)).strftime("%Y%m%d")
data = ak.stock_zh_a_hist(symbol="000001",period='daily',start_date=start_date,end_date=end_date,adjust="")
data = data.rename(columns={
    '日期': 'date',
    '开盘': 'open_price',
    '收盘': 'close_price',
    '最高': 'high_price',
    '最低': 'low_price',
    '成交量': 'volume',
    '成交额': 'turnover',
    '振幅': 'amplitude',
    '涨跌幅': 'price_change_pct',
    '涨跌额': 'price_change_amt',
    '换手率': 'turnover_rate'
})
data['stock_id']="000001"
data


,date,open_price,close_price,high_price,low_price,volume,turnover,amplitude,price_change_pct,price_change_amt,turnover_rate,stock_id
0,2023-11-27,10.09,10.01,10.09,9.97,836188,8.362684e+08,1.19,-0.89,-0.09,0.43,000001


In [31]:
from datetime import datetime
now = datetime.now()
now


AttributeError: 'datetime.datetime' object has no attribute 'dt'